In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.3 MB/s eta 0:00:00


In [ ]:
import json
import pdfplumber
import pandas as pd
import re
from typing import List, Dict
from unidecode import unidecode

In [ ]:
url = 'https://folger-main-site-assets.s3.amazonaws.com/uploads/2022/11/julius-caesar_PDF_FolgerShakespeare.pdf'
pdf_filename = 'julius_caesar.pdf'
!wget -O {pdf_filename} {url}

--2025-11-22 15:11:12--  https://folger-main-site-assets.s3.amazonaws.com/uploads/2022/11/julius-caesar_PDF_FolgerShakespeare.pdf
Resolving folger-main-site-assets.s3.amazonaws.com (folger-main-site-assets.s3.amazonaws.com)... 3.5.25.23, 16.15.176.200, 54.231.129.161, ...
Connecting to folger-main-site-assets.s3.amazonaws.com (folger-main-site-assets.s3.amazonaws.com)|3.5.25.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 617669 (603K) [application/pdf]
Saving to: ‘julius_caesar.pdf’

julius_caesar.pdf   100%[===================>] 603.19K  --.-KB/s    in 0.1s    

2025-11-22 15:11:12 (6.14 MB/s) - ‘julius_caesar.pdf’ saved [617669/617669]



In [ ]:
pages_data_plumber = []

with pdfplumber.open(pdf_filename) as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        pages_data_plumber.append({
            'page': i + 1,  # Page numbers are 1-based
            'raw': text
        })

output_json_filename_plumber = 'julius-caesar.json'
with open(output_json_filename_plumber, 'w', encoding='utf-8') as f:
    json.dump(pages_data_plumber, f, ensure_ascii=False, indent=2)

print(f"Extracted text from {len(pages_data_plumber)} pages using pdfplumber and saved to '{output_json_filename_plumber}'")

Extracted text from 106 pages using pdfplumber and saved to 'julius-caesar.json'


In [ ]:
INPUT_PATH = '/content/julius-caesar.json'

In [ ]:
CHARACTERS = {
    'FLAVIUS', 'MARULLUS', 'CARPENTER', 'COBBLER', 'CAESAR', 'CALPHURNIA',
    'BRUTUS', 'PORTIA', 'LUCIUS', 'CASSIUS', 'CASCA', 'CINNA', 'DECIUS',
    'LIGARIUS', 'METELLUS', 'CIMBER', 'TREBONIUS', 'CICERO', 'PUBLIUS',
    'POPILIUS', 'LENA', 'ANTONY', 'LEPIDUS', 'OCTAVIUS', 'SERVANT',
    'SOOTHSAYER', 'ARTEMIDORUS', 'LUCILIUS', 'TITINIUS', 'MESSALA',
    'VARRO', 'CLAUDIUS', 'CATO', 'STRATO', 'VOLUMNIUS', 'DARDANUS',
    'CLITUS', 'PINDARUS', 'FIRST', 'SECOND', 'THIRD', 'FOURTH', 'BOTH',
    'PLEBEIAN', 'PLEBEIANS', 'SOLDIER', 'SOLDIERS', 'MESSENGER', 'POET',
    'COMMONER', 'COMMONERS', 'CITIZENS', 'SENATORS', 'ALL'
}

In [ ]:
def clean_text(text: str) -> str:
    """Clean raw text by removing FTLN numbers and normalizing whitespace"""
    # Remove FTLN line numbers
    text = re.sub(r'FTLN\s+\d+', '', text)
    # Remove page headers like "11 Julius Caesar ACT 1. SC. 1"
    text = re.sub(r'\d+\s+Julius Caesar\s+ACT\s+\d+\.\s+SC\.\s+\d+', '', text)
    # Remove standalone numbers (line numbers)
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text)
    # Remove stage directions in brackets
    text = re.sub(r'\[.*?\]', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def is_valid_speaker(name: str) -> bool:
    name = name.upper().strip()
    return any(char in name or name in char for char in CHARACTERS)

In [ ]:
all_speeches = []
current_act = None
current_scene = None

print(f"Parsing Julius Caesar from {INPUT_PATH}")

with open(INPUT_PATH, 'r', encoding='utf-8') as f:
    pages = json.load(f)


for page_obj in pages:
    page_num = page_obj.get('page', 0)

    # Skip introductory pages
    if page_num < 9:
        continue

    # Clean the text
    text = clean_text(page_obj.get('raw', ''))

    # --- Detect Structure (Act/Scene) ---
    act_match = re.search(r'ACT\s+(\d+)', text, re.IGNORECASE)
    if act_match:
        current_act = int(act_match.group(1))

    scene_match = re.search(r'Scene\s+(\d+)', text, re.IGNORECASE)
    if scene_match:
        current_scene = int(scene_match.group(1))

    if not current_act:
        continue

    pattern = r'\b([A-Z][A-Z\s]{2,25}?)\b\s+([A-Z][a-z]|\bO\b|\bAy\b|\bNo\b|\bWhat\b|\bWhy\b|\bHow\b|\bI\b)'

    matches = list(re.finditer(pattern, text))

    for i, match in enumerate(matches):
        speaker = match.group(1).strip()

        if not is_valid_speaker(speaker):
            continue

        start_pos = match.start(2)
        end_pos = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        dialogue = text[start_pos:end_pos].strip()

        dialogue = re.sub(r'\b(Enter|Exit|Exeunt|Re-enter|Aside|They exit|He exits|She exits|All exit).*?\.', '', dialogue, flags=re.IGNORECASE)
        dialogue = re.sub(r'\(.*\)', '', dialogue)
        dialogue = ' '.join(dialogue.split())

        if len(dialogue) > 20 and not dialogue.startswith(('ACT', 'Scene', 'Enter', 'Exit')):
            all_speeches.append({
                'act': current_act,
                'scene': current_scene,
                'speaker': speaker,
                'text': dialogue
            })

print("Processing complete.")

Parsing Julius Caesar from /content/julius-caesar.json
Processing complete.


In [ ]:
df = pd.DataFrame(all_speeches)

df.to_json("dialogue_chunks.json", orient="records", indent=2)
print("\nData saved to 'dialogue_chunks.json'")


Data saved to 'dialogue_chunks.json'


In [ ]:
# def clean_unicode_and_numbers(text: str) -> str:
#     """Removes all Unicode characters and numbers from a string."""
#     # Remove numbers
#     text = re.sub(r'\d+', '', text)
#     # Remove non-ASCII characters (often covers most 'unicode' characters in this context)
#     text = text.encode('ascii', 'ignore').decode('ascii')
#     # Normalize whitespace after removals
#     text = ' '.join(text.split())
#     return text.strip()

# df['text'] = df['text'].apply(clean_unicode_and_numbers)

In [ ]:
def transliterate_and_clean_text(text: str) -> str:
    """Transliterates unicode to ASCII, then removes numbers and normalizes whitespace."""
    # Transliterate Unicode characters to ASCII equivalents
    text = unidecode(text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Normalize whitespace
    text = ' '.join(text.split())
    return text.strip()

In [ ]:
df['text'] = df['text'].apply(transliterate_and_clean_text)

In [ ]:
df.to_json("dialogue.json", orient="records", indent=2)
print("\nData (including cleaned text) saved again to 'dialogue.json'")


Data (including cleaned text) saved again to 'dialogue.json'
